In [18]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, Event
import pandas
import plotly.graph_objs as go
import time
import numpy as np
from scipy.sparse.linalg import svds
from surprise import Reader, Dataset, SVD, evaluate


dr = pandas.read_csv('ratings.csv')
dm = pandas.read_csv('movies.csv')





In [19]:
dr.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [20]:
n_users = dr.userId.unique().shape[0]
n_movies = dr.movieId.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 610 | Number of movies = 9724


In [21]:
mov_ids = dm['movieId']
mov_dict = {}
i=1
for id1 in mov_ids:
    mov_dict[id1]=i
    i += 1
len( mov_dict)

9742

In [22]:
# dr.replace({"movieId": mov_dict})

In [23]:
rats = dr.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
rats.head()


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
A = rats.as_matrix()
A


C:\Users\anik\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [25]:
users_average_rating = np.mean(A, axis = 1)
# users_average_rating
# len(users_average_rating)

In [26]:
rat_diff = A - users_average_rating.reshape(-1,1)
# rat_diff

In [27]:
# sparsity = round(1.0 - len(dr) / float(n_users * n_movies), 3)
# print ('The sparsity level of MovieLens1M dataset is ' +  str(sparsity * 100) + '%')

In [28]:
U, sigma, Vt = svds(rat_diff, k = 20)



In [29]:
sigma_mat = np.diag(sigma)

# sigma_mat

In [30]:
rat_predicted = np.dot(np.dot(U, sigma_mat), Vt) + users_average_rating.reshape(-1, 1)

user_id = 10
rec_mov = []


rat_predicted_user = rat_predicted[user_id,:]

# rat_predicted_user_list = rat_predicted_user.tolist()

# rat_predicted_user_list = rat_predicted_user


dp = dr.loc[dr['userId'] == int(user_id)]

user_mvs_ids = dp.iloc[:,1].tolist()


for i in range(0,len(rat_predicted_user)):
    if i in user_mvs_ids:
        rat_predicted_user[i] -= 100
        
rat_predicted_user



array([ 1.3008017 ,  0.7303839 ,  0.41577364, ...,  0.00227383,
        0.00227383, -0.00370914])

In [31]:
top_5_idx = np.argsort(-rat_predicted_user)[-5:]

top_5_idx
top_5_id_list = top_5_idx.tolist()

In [32]:
rec_mov_name = []

for mov_id in top_5_id_list:
    
    dummy = dm.loc[dm['movieId'] == mov_id]
    
    rec_mov_name.append(dummy.iloc[0,1])
    print(dm.loc[dm['movieId'] == mov_id])

# rec_mov_df = dm.loc[dm['movieId'] in top_5_id_list]

rec_mov_name

      movieId                      title          genres
3986     5620  Sweet Home Alabama (2002)  Comedy|Romance
      movieId                     title          genres
4131     5943  Maid in Manhattan (2002)  Comedy|Romance
      movieId                title          genres
2007     2671  Notting Hill (1999)  Comedy|Romance
     movieId              title         genres
694      912  Casablanca (1942)  Drama|Romance
      movieId               title                genres
3831     5377  About a Boy (2002)  Comedy|Drama|Romance


['Sweet Home Alabama (2002)',
 'Maid in Manhattan (2002)',
 'Notting Hill (1999)',
 'Casablanca (1942)',
 'About a Boy (2002)']

In [33]:
#model evaluation using surprise

reader = Reader()
data = Dataset.load_from_df(dr[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

svd = SVD()
evaluate(svd, data, measures=['RMSE'])

C:\Users\anik\Anaconda3\lib\site-packages\surprise\evaluate.py:66: UserWarning:

The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.

C:\Users\anik\Anaconda3\lib\site-packages\surprise\dataset.py:193: UserWarning:

Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 



Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.8706
------------
Fold 2
RMSE: 0.8719
------------
Fold 3
RMSE: 0.8745
------------
Fold 4
RMSE: 0.8725
------------
Fold 5
RMSE: 0.8714
------------
------------
Mean RMSE: 0.8722
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.8706237881833898,
                             0.8718706673707995,
                             0.8744841338618948,
                             0.8724587596963788,
                             0.8713927809668955]})

In [36]:
print(rat_predicted.shape)

(610, 9724)
